## Random Forest Classifiers
<ins>R</ins>andom <ins>F</ins>orest <ins>C</ins>lassifiers use an ensemble method meaning models are created by combining the result of many individual decision tree models. Decision trees classify a set of features by combining (many) layers of sub-classifications. RFCs train small decision trees, that is ones with only a few layers, and use the most popular result of all the trees to classify the set of features. 

RFCs can produce powerful models and are particulary good at working with categorical. They do not scale well with large amounts of data. Another disadvantage is they can train to favor some features more than others. In  many situations RFC models perform as good or better than other types of models. 

## Using RFC on the Credit Card Churn Data
The credit card data includes both demographic and 

## Setup
The mechlearn file was writen to easily compute the accuracy score and 'area under the curve' for a given model. The 'auct' function splits and scales the proved data sets, creates a model with them, generates an receiver/operater curve and returns the false positive and true posititves rates as well the area under the curve. The 'acct' fuinction splits and scales the proved data sets, creates a model with them and scores the accuracy of the model on the testing set.

In [ ]:
# import dependencies
import pandas as pd
from mechlearn import split_and_scale as ss
from mechlearn import auc_test as auct
from mechlearn import acc_test as acct
from mechlearn import feature_importances as fi

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

The 'Resources/X.csv' and 'Resources/y.csv' files were generated in the etl_workflow.ipynb and hold the cleaned data and target values for the entire dataset.

In [ ]:
# load data (X) and target (y) into dataframes
X = pd.read_csv('../Resources/X.csv')
y = pd.read_csv('../Resources/y.csv')

## Evaluate Model Using All Features

In [ ]:
features = X.columns
print(f'Feature names: {features}')
print(f'Number of Features: {len(features)}')

In [ ]:
print('Random forest classifiier on full data')
print(f'Accuracy: {acct(X, y, model="rfc")}')
print(f'Area under curve: {auct(X, y, model="rfc")}')

In [ ]:
plot = pd.read_csv('../Outputs/RandomForestClassifier_ROC.csv')
plt.plot(plot['fpr'], plot['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier Using All Features')
plt.show()

In [ ]:
feature_importances = fi(X, y, model='rfc')
ticks = np.arange(len(X.columns))
fig, ax = plt.subplots()
ax.barh(ticks, feature_importances)
ax.set_yticks(ticks, labels=X.columns)
fig.set_size_inches(15, 15)
ax.set_xlabel('Feature Importances')
ax.set_title('Feature Importances for Random Forest Classifier')
plt.show()
pd.DataFrame({'Features': features, 'Feature_Importances': feature_importances}).to_csv('../Outputs/RandomForestClassifier_FI.csv')

## Evaluate Model with Just Demographics Data

In [ ]:
Xdem = pd.read_csv('../Resources/X-dem.csv')

In [ ]:
features_dem = Xdem.columns.values
print(f'Feature names: {features_dem}')
print(f'Number of Features: {len(features_dem)}')

In [ ]:
print('Random forest classifiier on demographic data only')
print(f'Accuracy: {acct(Xdem, y, model="rfc")}')
print(f'Area under curve: {auct(Xdem, y, model="rfc", data_set="X-dem")}')

In [ ]:
plot_dem = pd.read_csv('../Outputs/X-dem_RandomForestClassifier_ROC.csv')
plt.plot(plot_dem['fpr'], plot_dem['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier Using Demographics')
plt.show()

In [ ]:
feature_importances_dem = fi(Xd, y, model='rfc')
ticks = np.arange(len(features_dem))
fig, ax = plt.subplots()
ax.barh(ticks, feature_importances_dem)
ax.set_yticks(ticks, labels=features_dem)
fig.set_size_inches(15, 15)
ax.set_xlabel('Feature Importances')
ax.set_title('Feature Importances for Random Forest Classifier Using Demographics')
plt.show()
pd.DataFrame({'Features': features_dem, 'Feature_Importances': feature_importances_dem}).to_csv('../Outputs/X-dem_RandomForestClassifier_FI.csv')

## Evaluate Model with PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier as RFC
from mechlearn import roc

_Xs, Xs_, _ypca, ypca_ = ss(X, y)
pca = PCA(n_components=21)
_Xspca, Xspca_ = pca.fit_transform(_Xs), pca.transform(Xs_)
print(f'Variance ratio of components: {[round(variance,2) for variance in pca.explained_variance_ratio_]}')
print(f'Number of components: 21')

In [ ]:
rfc_pca = RFC()
rfc_pca.fit(_Xspca, _ypca.to_numpy().ravel())
score = rfc_pca.score(Xspca_, ypca_)
auc = roc(Xspca_, ypca_, rfc_pca, 100, area=True, save_path='../Outputs/X-pca_RandomForestClassifier_ROC.csv')
print('Random forest classifiier with PCA')
print(f'Accuracy: {score}')
print(f'Area under curve: {auc}')

In [ ]:
plot_pca = pd.read_csv('../Outputs/X-pca_RandomForestClassifier_ROC.csv')
plt.plot(plot_pca['fpr'], plot_pca['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier with PCA')
plt.show()

In [ ]:
rfc_dem = RFC()
rfc_dem.fit(Xdem, y)